In [11]:
#!/usr/bin/env python3
"""
Automated Phylogenetic Tree Builder
-----------------------------------

Paste your inputs in the section below, then just run:
    python build_phylo_automation_simple.py

This script will:
  1. Extract your seed & outgroup protein sequences by accession
  2. BLAST each within their respective family FASTAs
  3. Cluster the seed results at 90% identity using CD-HIT
  4. Combine both FASTAs
  5. Align with MAFFT
  6. Build a FastTree

Requirements (must be installed and in PATH):
    makeblastdb, blastp, blastdbcmd, cd-hit, mafft, fasttree
"""

import subprocess
import sys
import shutil
from pathlib import Path
from Bio import SeqIO

####################################
 #INPUTS#
####################################

# Seed (the protein of interest)
SEED_ACCESSION = "G2G6S2"              # e.g. "A0A123XYZ1"
SEED_FAMILY_FASTA = "protein-matching-IPR014184.fasta"

# Outgroup (reference/other family)
OUTGROUP_ACCESSION = "A0ABR5CPA3"      # e.g. "WP_0987654321"
OUTGROUP_FAMILY_FASTA = "protein-matching-IPR014183.fasta"

# Parameters
SEED_TOP_HITS = 1000
OUTGROUP_TOP_HITS = 100
THREADS = 4
CLUSTER_IDENTITY = 0.9  # 90%

# Any extra fasta files to include in the final alignment/tree
additional_fastas = [
    "P46154.fasta",
    "Q9HTE3.fasta",
    "A0A0H3KP42.fasta"
]

####################################


#Making sure the right stuff is installed
def check_exec(name):
    if shutil.which(name) is None:
        sys.exit(f" '{name}' not found ")

# When we open a fasta file, we need te accession number and sequence. If we cant find it, we stop.
def extract_record(fasta, acc):
    with open(fasta) as f:
        for rec in SeqIO.parse(f, "fasta"):
            if acc in rec.id or acc in rec.description:
                return rec
    sys.exit(f" {acc} not found in {fasta}")

#This writes the file in consistent fasta format (>ACCESSION...)
def write_record(rec, filename):
    with open(filename, "w") as f:
        SeqIO.write(rec, f, "fasta")

#This tells us what command is currently being run, opens the output file being made, runs the command, checks it worked
def run_cmd(cmd, output=None):
    print("▶", " ".join(cmd))
    if output:
        with open(output, "w") as f:
            subprocess.run(cmd, check=True, stdout=f)
    else:
        subprocess.run(cmd, check=True)

# Check tools
for tool in ["makeblastdb", "blastp", "blastdbcmd", "cd-hit", "mafft", "fasttree"]:
    check_exec(tool)

# Extract sequences
seed_rec = extract_record(SEED_FAMILY_FASTA, SEED_ACCESSION)
out_rec = extract_record(OUTGROUP_FAMILY_FASTA, OUTGROUP_ACCESSION)

seed_query = f"seed_{SEED_ACCESSION}.fasta"
out_query = f"out_{OUTGROUP_ACCESSION}.fasta"
write_record(seed_rec, seed_query)
write_record(out_rec, out_query)

print(f"✅ Extracted {SEED_ACCESSION} and {OUTGROUP_ACCESSION}")

# Build BLAST DBs
seed_db = str(Path(SEED_FAMILY_FASTA).resolve().with_suffix(''))  # Remove extension
out_db = str(Path(OUTGROUP_FAMILY_FASTA).resolve().with_suffix(''))  # Remove extension

# Force rebuild BLAST DBs
print("Building BLAST databases...")
run_cmd(["makeblastdb", "-in", SEED_FAMILY_FASTA, "-dbtype", "prot", "-out", seed_db, "-parse_seqids"])
run_cmd(["makeblastdb", "-in", OUTGROUP_FAMILY_FASTA, "-dbtype", "prot", "-out", out_db, "-parse_seqids"])

# Verify database files exist
required_extensions = ['.phr', '.pin', '.psq']
for ext in required_extensions:
    if not Path(seed_db + ext).exists():
        sys.exit(f" BLAST database file missing: {seed_db + ext}")
    if not Path(out_db + ext).exists():
        sys.exit(f" BLAST database file missing: {out_db + ext}")

# BLAST seed
seed_hits = f"seed_hits.txt"
cmd = [
    "blastp", "-query", seed_query, "-db", seed_db,
    "-outfmt", "6 sacc bitscore", "-max_target_seqs", str(SEED_TOP_HITS), "-evalue", "1e-5"
]
res = subprocess.run(cmd, capture_output=True, text=True, check=True)
hits = [l.split()[0] for l in res.stdout.strip().splitlines()]
Path(seed_hits).write_text("\n".join(hits))
print(f" Retrieved {len(hits)} top seed hits")

# Fetch seed hit sequences
seed_fasta = f"seed_{SEED_ACCESSION}_top{SEED_TOP_HITS}.fasta"
run_cmd(["blastdbcmd", "-db", seed_db, "-entry_batch", seed_hits], output=seed_fasta)

# Cluster seed hits
seed_clustered = f"{seed_fasta}_cdhit90.fasta"
run_cmd([
    "cd-hit", "-i", seed_fasta, "-o", seed_clustered,
    "-c", str(CLUSTER_IDENTITY), "-n", "5", "-T", str(THREADS), "-M", "0"
])
print(f" Clustered seed hits at {CLUSTER_IDENTITY*100:.0f}% identity")

# BLAST outgroup
out_hits = f"out_hits.txt"
cmd = [
    "blastp", "-query", out_query, "-db", out_db,
    "-outfmt", "6 sacc bitscore", "-max_target_seqs", str(OUTGROUP_TOP_HITS), "-evalue", "1e-5"
]
res = subprocess.run(cmd, capture_output=True, text=True, check=True)
hits = [l.split()[0] for l in res.stdout.strip().splitlines()]
Path(out_hits).write_text("\n".join(hits))
print(f" Retrieved {len(hits)} top outgroup hits")

# Fetch outgroup sequences
out_fasta = f"out_{OUTGROUP_ACCESSION}_top{OUTGROUP_TOP_HITS}.fasta"
run_cmd(["blastdbcmd", "-db", out_db, "-entry_batch", out_hits], output=out_fasta)

# Combine FASTAs
combined = f"combined_{SEED_ACCESSION}_{OUTGROUP_ACCESSION}.fasta"
with open(combined, "w") as out:
    for f in [seed_clustered, out_fasta]:
        for rec in SeqIO.parse(f, "fasta"):
            out.write(f">{rec.id}\n{str(rec.seq)}\n")
print(f" Combined FASTA written to {combined}")

# Append additional fasta files in the directory to the combined fasta file

def append_fastas_to_combined(combined_fasta, fasta_files):
    with open(combined_fasta, "a") as out:
        for fasta in fasta_files:
            for rec in SeqIO.parse(fasta, "fasta"):
                out.write(f">{rec.id}\n{str(rec.seq)}\n")
    print(f"Appended {len(fasta_files)} fasta files to {combined_fasta}")


append_fastas_to_combined(combined, additional_fastas)

# MAFFT alignment
aligned = f"{combined}_aligned.fasta"
run_cmd(["mafft", "--auto", "--thread", str(THREADS), combined], output=aligned)
print(f" Alignment done: {aligned}")

# FastTree
tree_file = f"{combined}_with_{additional_fastas}.nwk"
run_cmd(["fasttree", "-gamma", "-wag", aligned], output=tree_file)
print(f" Phylogenetic tree built: {tree_file}")

print("\n All steps completed successfully!")

✅ Extracted G2G6S2 and A0ABR5CPA3
Building BLAST databases...
▶ makeblastdb -in protein-matching-IPR014184.fasta -dbtype prot -out /home/tommyor99/protein-matching-IPR014184 -parse_seqids


Building a new DB, current time: 10/29/2025 14:29:18
New DB name:   /home/tommyor99/protein-matching-IPR014184
New DB title:  protein-matching-IPR014184.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/tommyor99/protein-matching-IPR014184
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 3384 sequences in 0.065769 seconds.
▶ makeblastdb -in protein-matching-IPR014183.fasta -dbtype prot -out /home/tommyor99/protein-matching-IPR014183 -parse_seqids


Building a new DB, current time: 10/29/2025 14:29:18
New DB name:   /home/tommyor99/protein-matching-IPR014183
New DB title:  protein-matching-IPR014183.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/tommyor99/protein-matching-IPR014183
Keep MBits: T
Maximum fi

nthread = 4
nthreadpair = 4
nthreadtb = 4
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  301 / 376 (thread    2)
done.

Constructing a UPGMA tree (efffree=0) ... 
  370 / 376
done.

Progressive alignment 1/2... 
STEP   349 / 375 (thread    3)
Reallocating..done. *alloclen = 1911
STEP   375 / 375 (thread    2)
done.

Making a distance matrix from msa.. 
  300 / 376 (thread    2)
done.

Constructing a UPGMA tree (efffree=1) ... 
  370 / 376
done.

Progressive alignment 2/2... 
STEP   342 / 375 (thread    3)
Reallocating..done. *alloclen = 1920
STEP   375 / 375 (thread    3)
done.

disttbfast (aa) Version 7.525
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
4 thread(s)

distout=h
rescale = 1
  300 / 376 (thread    0)dndpre (aa) Version 7.525
alg=X, model=BLOSUM62, 1.53, +0.12, -0.00, noshift, amax=0.0
4 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 4
randomseed = 0
blosum 62 / kimu

 Alignment done: combined_G2G6S2_A0ABR5CPA3.fasta_aligned.fasta
▶ fasttree -gamma -wag combined_G2G6S2_A0ABR5CPA3.fasta_aligned.fasta


      0.10 seconds: Joined    200 of    359
Initial topology in 0.19 seconds
Refining topology: 34 rounds ME-NNIs, 2 rounds ME-SPRs, 17 rounds ML-NNIs
      0.20 seconds: ME NNI round 1 of 34, 301 of 360 splits, 69 changes (max delta 0.022)
      0.36 seconds: SPR round   1 of   2, 101 of 722 nodes
      0.56 seconds: SPR round   1 of   2, 301 of 722 nodes
      0.69 seconds: SPR round   1 of   2, 501 of 722 nodes
      0.89 seconds: SPR round   1 of   2, 701 of 722 nodes
      1.02 seconds: SPR round   2 of   2, 101 of 722 nodes
      1.15 seconds: SPR round   2 of   2, 301 of 722 nodes
      1.33 seconds: SPR round   2 of   2, 501 of 722 nodes
      1.51 seconds: SPR round   2 of   2, 701 of 722 nodes
Total branch-length 22.766 after 1.60 sec
      1.80 seconds: ML Lengths 101 of 360 splits
      2.03 seconds: ML Lengths 201 of 360 splits
      2.25 seconds: ML Lengths 301 of 360 splits
      2.38 seconds: ML NNI round 1 of 17, 1 of 360 splits
      3.13 seconds: ML NNI round 1 of 17

 Phylogenetic tree built: combined_G2G6S2_A0ABR5CPA3.fasta_with_['P46154.fasta', 'Q9HTE3.fasta', 'A0A0H3KP42.fasta'].nwk

 All steps completed successfully!


     18.79 seconds: Optimizing alpha round 1
Gamma(20) LogLk = -58912.647 alpha = 0.395 rescaling lengths by 1.328
Total time: 18.80 seconds Unique: 362/376 Bad splits: 0/359
